In [ ]:
from typing import Protocol, runtime_checkable, TypeVar, Set, Tuple
from queue import Queue
from transitions import Machine

#################################################################
#  Atlas Display Roster                                         #
#    |     |      |                Inputs                       #
#   Map Widget Entity                |                          #
#    \    |    /                     |                          #
#    Game States ___ SMs(AI) __ +Game Event(AI) __ Handle(AI)   #
#         |                          |               |          #
#   Action Perform ______________ Reactions   State Event Queue #      
#         |                                          |          #                 
#   "Dispatch" Thread                         "Handle" Thread   #
#         |                                          |          #
#    State Action Queue                         Event Trigger   #
#         |                                          |          #
#         |___________________+Game Action(AI) __ Dispatch(AI)  #
#                                                               #
# ############################################################### 

class GameState:
    actions: Queue
    events: Queue
    roster: object
    atlas: object
    display: object


@runtime_checkable
class GameStateObject(Protocol):
    state: GameState | None

@runtime_checkable
class StateTransitionObject(GameStateObject, Protocol):
    transformer: object | None
    
@runtime_checkable
class GameAction(StateTransitionObject, Protocol):
    """ The Game Action is created by the Dispatcher and is sent to the State Actions queue.
    Actions have a 'perform' method that wraps the execution of a Dispatcher method. When the GameState pulls the Game Action from the queue, it
    calls the 'perform' method to execute the Handler method."""

    state: GameState | None
    transformer: object | None # This should be set to the Handler that will handle any associated reaction events.
    
    def __init__(self) -> None:
        self.state = None
        self.transformer = None
    
    def perform(self) -> None:
        ...

@runtime_checkable
class GameEvent(StateTransitionObject,Protocol):
    """ The Game Event is created by a Handler and sent to the State Events queue. 
    Game Events have a 'trigger' method that wraps the execution of an associated 
    Dispatcher method. When the GameState pulls the Game Event from the queue, it
    calls the 'trigger' method to execute the Dispatcher method."""
    state: GameState | None
    transformer: object | None # This should be set to the Dispatcher that will dispatch the associated action.

    def __init__(self) -> None:
        self.state = None
        self.transformer = None
        
    def trigger(self) -> None:
        ...

@runtime_checkable
class StateTransformer(Protocol):
    """The StateFormer Protocol is a mixin class that has a 'transform' and 'enqueue' methods. The 'transform' method
      provides sets the attributes of StateTransitionObject instances with the Game State data required
      by the STO methods."""
    templates: Set[Tuple[str, StateTransitionObject]]
    machine: Machine
    
    T = TypeVar('T', bound=StateTransitionObject)

    def _enqueue(self, input_item: StateTransitionObject)  -> bool:
        
        if input_item.state is not None:
            output_item: StateTransitionObject | None
            output_items: Queue[StateTransitionObject]

            if isinstance(input_item, GameEvent):
                output_items = input_item.state.actions
            else:
                output_items = input_item.state.events

            try:
                output_item_name = type(input_item).__name__.lower()
                output_item = self.get_template(output_item_name)

                if output_item:
                    transformed_item = self.transform(output_item, input_item.state)
                    output_items.put(transformed_item)
                    return True
                
                return False
        
            except Exception as e:
                raise e
            
        return False
    
    def get_template(self, name: str) -> StateTransitionObject | None:
        for template in self.templates:
            if template[0] == name:
                return template[1]
        return None
    
    def transform(self, item: T, state: GameState) -> T:
        item.state = state
        item.transformer = self
        return item





In [5]:
import tcod

class GameHandler(StateTransformer):
    """The GameHandler is responsible for tranforming Game Inputs and AI Actions into Game Events
    and sending them to the State Action Queue."""

    templates: Set[Tuple[str, StateTransitionObject]]
    machine: Machine

    def __init__(self):
        super().__init__()
        self.templates = {
                        ('noaction', NonEvent()),
                        ('nonevent', NoAction()),
                        }
        
    def dispatch(self, event: GameEvent) -> bool:
        return super()._enqueue(event)   
        
    def handle(self, event: GameAction | tcod.event.Event | None = None) -> bool:
        if event is not None:
            if isinstance(event, tcod.event.Event):
                # Handle tcod event conversion to GameEvent here if needed
                pass

            parsed_event: GameAction | GameEvent = event  # type: ignore

            return super()._enqueue(parsed_event)
        
        return False

class BaseEvent:
    state: GameState | None
    transformer: object | None

    def __init__(self, handler: GameHandler | None = None, state: GameState | None = None) -> None:
        self.state = state
        self.transformer = handler
    
    def trigger(self) -> None:
        self.transformer.dispatch(self) # type: ignore


class NonEvent(BaseEvent):
    def __init__(self, handler: GameHandler | None = None, state: GameState | None = None) -> None:
        super().__init__(handler, state)

    
class BaseAction:
    state: GameState | None
    transformer: object | None
    
    def __init__(self, handler: GameHandler | None = None, state: GameState | None = None) -> None:
        self.state = state
        self.transformer = handler

    def perform(self) -> None:
        ...
    

class NoAction(BaseAction):
    
    def __init__(self, handler: GameHandler | None = None, state: GameState | None = None) -> None:
        super().__init__(handler, state)

    def perform(self) -> None:
        event = self.transformer.get_template('nonevent') # type: ignore
        self.transformer.handle(event) # type: ignore

        print("No action performed, no reaction events")


In [6]:
game_state = GameState()
game_state.actions = Queue()
game_state.events = Queue()

a = NonEvent()
a.state = game_state
a.transformer = GameHandler()
a.trigger()

next_action = game_state.actions.get_nowait()
next_action.perform()

next_event = game_state.events.get_nowait()
next_event.trigger()

next_action = game_state.actions.get_nowait()
next_action.perform()

next_event = game_state.events.get_nowait()
next_event.trigger()



No action performed, no reaction events
No action performed, no reaction events


In [7]:
game_state = GameState()
game_state.actions = Queue()
game_state.events = Queue()

handler = Handler()
dispatcher = Dispatcher()

past_action = NoAction()
handler.handle(past_action, game_state)

new_event = game_state.events.get_nowait()
new_event.trigger() # Action is sent to State Action Queue




NameError: name 'Handler' is not defined